In [10]:
# import networkx as nx
import numpy as np
from collections import deque
import heapq #used for a_star algorithm

# The root
root = "123704865"

# The worst possible root requiring 31 moves!
# root = "867254301"

In [11]:
class State:
    def __init__(self, value, parent=None, action=None, depth=0, g=None, h=None):
        """
        Parameters:
        value (numpy.ndarray): The state represented as a numpy array.
        parent (State): The parent state leading to this state.
        action (str): The action taken to reach this state from the parent.
        depth (int): used for BFS and DFS
        g and h (int): used for A star algorithm
        """
        if not isinstance(value, np.ndarray):
            raise ValueError("value must be a numpy array.")
        
        self.value = value
        self.parent = parent
        self.action = action
        self.depth = depth 
        self.g = g #This is the g of the state in the A star algorithm
        self.h = h #This is the h of the state in the A star algorithm

    def set_g(g):
        self.g=g
        
    def set_h(h):
        self.h=h

    # used for the min heap in a star algorithm
    def __lt__(self, other):
        return self.g + self.h < other.g + other.h

    def __repr__(self):
        if self.parent is None:  
            return f"State(value={self.value}, parent={self.parent}, action={self.action}, depth={self.depth})"
        else:
            return f"State(value={self.value}, parent={self.parent.value}, action={self.action}, depth={self.depth})"

    def copy(self):
        return State(value=np.copy(self.value), parent=self.parent, action=self.action, depth=self.depth)

In [12]:
def get_action_list(current_state):
    temp = []
    
    # check to see if the empty square is not in the third row
    if not(current_state.value[6] * current_state.value[7] * current_state.value[8] == 0):
        temp += "d"

    # check to see if the empty square is not in the third column
    if not(current_state.value[2] * current_state.value[5] * current_state.value[8] == 0):
        temp += "r"

    # check to see if the empty square is not in the first column
    if not(current_state.value[0] * current_state.value[3] * current_state.value[6] == 0):
        temp += "l"
        
    # check to see if the empty square is not in the first row
    if not(current_state.value[0] * current_state.value[1] * current_state.value[2] == 0):
        temp += "u"
        
    return temp

In [13]:
def apply_action(current_state, current_action):
    temp = State(value=current_state.value.copy(), parent=current_state, action=None, depth=current_state.depth + 1)
    index = np.where(temp.value == 0)[0][0]
    
    if(current_action == "r"):
        # Swap the elements
        temp.value[index], temp.value[index+1] = temp.value[index+1], temp.value[index]
        temp.action = "r"
        return temp

    if(current_action == "d"):
        # Swap the elements
        temp.value[index], temp.value[index+3] = temp.value[index+3], temp.value[index]
        temp.action = "d"
        return temp

    if(current_action == "l"):
        # Swap the elements
        temp.value[index], temp.value[index-1] = temp.value[index-1], temp.value[index]
        temp.action = "l"
        return temp

    if(current_action == "u"):
        # Swap the elements
        temp.value[index], temp.value[index-3] = temp.value[index-3], temp.value[index]
        temp.action = "u"
        return temp


In [14]:
def back_track(some_state):
    the_list = []
    temp = some_state
    while(temp.parent is not None):
        the_list.append(temp.value)
        the_list.append(temp.action)
        temp = temp.parent
    the_list.append(temp.value)
    return(the_list[::-1])

In [15]:
def bfs(root):
    # Convert the number to a NumPy array of its digits
    initial_value = np.array([int(digit) for digit in root])
    target_value = np.array([int(digit) for digit in "123456780"])
    
    initial_state = State(value=initial_value, parent=None, action=None, depth=0)
    
    bfs_queue = deque()
    bfs_queue.append(initial_state)
    
    visited = []
    visited.append(initial_value)
    
    steps = 0
    
    if(np.array_equal(initial_state.value, target_value)):
        print("The answer was found in 0 steps")
        return None
    
    else:    
        # I don't want to do too many steps that is why I have this here.
        while (steps < 10000):
            steps += 1
    
            if(len(bfs_queue) == 0):
                print("There are no answers to this problem. This is an invalid input.")
                return None
    
            current_state = bfs_queue.popleft().copy()
           
            for i in get_action_list(current_state):
                new_state = apply_action(current_state, i)
    
                if(np.array_equal(new_state.value, target_value)):
                    # the answer was found!
                    solution = back_track(new_state)
                    print(f"The answer was found in {int(len(solution)/2)} steps")
                    return solution
                    
                if not (any(np.array_equal(new_state.value, v) for v in visited)):
                    bfs_queue.append(new_state)
                    visited.append(new_state.value)

        print("The process is taking too long! I give up!")
        return None

In [16]:
bfs(root)

The answer was found in 8 steps


[array([1, 2, 3, 7, 0, 4, 8, 6, 5]),
 'r',
 array([1, 2, 3, 7, 4, 0, 8, 6, 5]),
 'd',
 array([1, 2, 3, 7, 4, 5, 8, 6, 0]),
 'l',
 array([1, 2, 3, 7, 4, 5, 8, 0, 6]),
 'l',
 array([1, 2, 3, 7, 4, 5, 0, 8, 6]),
 'u',
 array([1, 2, 3, 0, 4, 5, 7, 8, 6]),
 'r',
 array([1, 2, 3, 4, 0, 5, 7, 8, 6]),
 'r',
 array([1, 2, 3, 4, 5, 0, 7, 8, 6]),
 'd',
 array([1, 2, 3, 4, 5, 6, 7, 8, 0])]

In [17]:
def dfs_with_limit(root, depth_limit):
    # Convert the number to a NumPy array of its digits
    initial_value = np.array([int(digit) for digit in root])
    target_value = np.array([int(digit) for digit in "123456780"])
    
    initial_state = State(value=initial_value, parent=None, action=None, depth=0)
    
    dfs_queue = deque()
    dfs_queue.append(initial_state)
    
    visited = []
    
    steps = 0
    
    if(np.array_equal(initial_state.value, target_value)):
        print("The answer was found")
        return None
    
    else:    
        # I don't know how much this should be to garantee a result but it sure is a large number.
        while not (len(dfs_queue) == 0):
            steps += 1
            
            current_state = dfs_queue.pop().copy()
            if not (any(np.array_equal(current_state.value, v) for v in visited)):
                # print("here")
                visited.append(current_state.value)         
                for i in get_action_list(current_state):
                    new_state = apply_action(current_state, i)
                    if(new_state.depth < depth_limit):
                        dfs_queue.append(new_state)
        
                    if(np.array_equal(new_state.value, target_value)):
                        # the answer was found!
                        solution = back_track(new_state)
                        print("The answer was found")
                        return solution
        return None

In [18]:
dfs_with_limit(root, 3)

In [19]:
# iterative deepening depth-first search
def IDDFS(root, limit):
    for i in range(1, limit + 1):
        if not(dfs_with_limit(root, limit) is None):
            # the answer was found
            print(f"The answer was found at depth limit={limit}")
            return None
            
    print("No answer was found at up to this depth")

In [20]:
IDDFS(root,8)

The answer was found
The answer was found at depth limit=8


In [21]:
def bidirectional_bfs(root):
    # Convert the number to a NumPy array of its digits
    initial_value = np.array([int(digit) for digit in root])
    target_value = np.array([int(digit) for digit in "123456780"])
    
    initial_state = State(value=initial_value, parent=None, action=None, depth=0)
    target_state = State(value=target_value, parent=None, action=None, depth=0)
    
    bfs_queue1 = deque()
    bfs_queue1.append(initial_state)

    bfs_queue2 = deque()
    bfs_queue2.append(target_state)
    
    visited1 = []
    visited1.append(initial_state)

    visited2 = []
    visited2.append(target_state)
    
    steps = 0
    
    if(np.array_equal(initial_state.value, target_value)):
        print("The answer was found in 0 steps")
        return None
    
    else:    
        # I don't want to do too many steps that is why I have this here.
        while (steps < 10000):
            steps += 1
    
            if(len(bfs_queue1) == 0 or len(bfs_queue2) == 0):
                print("There are no answers to this problem. This is an invalid input.")
                return None
    
            current_state1 = bfs_queue1.popleft().copy()
            current_state2 = bfs_queue2.popleft().copy()
           
            for i in get_action_list(current_state1):
                new_state1 = apply_action(current_state1, i)
    
                # if(any(np.array_equal(new_state1.value, v) for v in visited2)):
                match1 = None
                for i in visited2:
                    if(np.array_equal(new_state1.value,i.value)):
                        match1=i
                
                if match1 is not None:
                    # The answer was found!
                    solution1 = back_track(new_state1)
                    solution2 = back_track(match1)[::-1]

                    direction_mapping = {'u': 'd', 'd': 'u', 'l': 'r', 'r': 'l'}

                    # Iterate through the list and apply the mapping
                    for i in range(len(solution2)):
                        if isinstance(solution2[i], str) and solution2[i] in direction_mapping:
                            solution2[i] = direction_mapping[solution2[i]]

                    return solution1 + solution2[1::]

                found1=False
                for i in visited1:
                    if(np.array_equal(new_state1.value, i.value)):
                        found1=True
                        break
                if not(found1):
                    bfs_queue1.append(new_state1)
                    visited1.append(new_state1)
                found1=False

            # the reverse
            for i in get_action_list(current_state2):
                new_state2 = apply_action(current_state2, i)
    
                # if(any(np.array_equal(new_state2.value, v) for v in visited1)):
                match2 = None
                for j in visited1:
                    if(np.array_equal(new_state2.value,j.value)):
                        match1=j
                
                # next((v for v in visited1 if np.array_equal(new_state2.value, v)), None)
                if match2 is not None:
                    # the answer was found!
                    solution1 = back_track(match2)
                    solution2= back_track(new_state2)[::-1]

                    direction_mapping = {'u': 'd', 'd': 'u', 'l': 'r', 'r': 'l'}

                    # Iterate through the list and apply the mapping
                    for i in range(len(solution2)):
                        if isinstance(solution2[i], str) and solution2[i] in direction_mapping:
                            solution2[i] = direction_mapping[solution2[i]]
                            
                    return solution1 + solution2[1::]                

                found2=False
                for i in visited2:
                    if(np.array_equal(new_state2.value, i.value)):
                        found2=True
                        break
                if not(found2):
                    bfs_queue2.append(new_state2)
                    visited2.append(new_state2)
                found2=False

        print("The process is taking too long! I give up!")  
    print(0)

In [22]:
bidirectional_bfs(root)

[array([1, 2, 3, 7, 0, 4, 8, 6, 5]),
 'r',
 array([1, 2, 3, 7, 4, 0, 8, 6, 5]),
 'd',
 array([1, 2, 3, 7, 4, 5, 8, 6, 0]),
 'l',
 array([1, 2, 3, 7, 4, 5, 8, 0, 6]),
 'l',
 array([1, 2, 3, 7, 4, 5, 0, 8, 6]),
 'u',
 array([1, 2, 3, 0, 4, 5, 7, 8, 6]),
 'r',
 array([1, 2, 3, 4, 0, 5, 7, 8, 6]),
 'r',
 array([1, 2, 3, 4, 5, 0, 7, 8, 6]),
 'd',
 array([1, 2, 3, 4, 5, 6, 7, 8, 0])]

In [23]:
def calculate_coordinate_index(index):
    point = [index // 3, index % 3]
    return point

def calculate_coordinate_number(number):
    point = [(number - 1) // 3, (number - 1) % 3]
    return point

def calculate_distance(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def calculate_h(my_state):
    h = 0
    for i in range (len(my_state.value)):
        # print(i)
        if (my_state.value[i] == 0):
            h+= calculate_distance(calculate_coordinate_number(9),calculate_coordinate_index(i))
        else:
            h+= calculate_distance(calculate_coordinate_number(my_state.value[i]),calculate_coordinate_index(i))
        # print(h)
    return h
    
def a_star(root):
    initial_value = np.array([int(digit) for digit in root])
    target_value = np.array([int(digit) for digit in "123456780"])
    
    initial_state = State(value=initial_value, parent=None, action=None, depth=0)
    initial_state.h = calculate_h(initial_state)
    initial_state.g = 0

    min_heap = []
    heapq.heappush(min_heap, initial_state) 
    
    visited = [initial_state]
    while not (len(min_heap)==0):
        temp = heapq.heappop(min_heap)
        if(np.array_equal(temp.value, target_value)):
            print("The answer was found")
            # print(back_track(temp))
            return back_track(temp)
        for i in get_action_list(temp):
            new_state = apply_action(temp, i)
            new_state.h = calculate_h(new_state)
            new_state.g = temp.g + 1
            heapq.heappush(min_heap, new_state)

In [24]:
print(a_star(root)[1::2]) #prints only the moves

The answer was found
['r', 'd', 'l', 'l', 'u', 'r', 'r', 'd']


In [25]:
import tkinter as tk
from random import shuffle

class SlidingPuzzle:
    def __init__(self, frame, caption_text=""):
        self.frame = frame
        self.frame.title("3x3 Sliding Puzzle")
        self.grid_size = 3
        # self.tiles = [i for i in range(1, self.grid_size ** 2)] + [0]  # Numbers + empty tile
        # self.empty_pos = self.grid_size ** 2 - 1  # Initial empty position
        self.tiles = [int(digit) for digit in root]
        self.empty_pos = root.find('0')
        self.buttons = []
        
        self.caption_text = caption_text  # The caption text
        self.setup_ui()
        self.bind_keys()

    def setup_ui(self):
        for i in range(self.grid_size ** 2):
            btn = tk.Button(self.frame, text=str(self.tiles[i]) if self.tiles[i] else "", font=("Arial", 24),
                            width=4, height=2, command=lambda i=i: self.move_tile(i))
            btn.grid(row=i // self.grid_size, column=i % self.grid_size)
            self.buttons.append(btn)

        # Add the caption label
        self.caption_label = tk.Label(self.frame, text=self.caption_text, font=("Arial", 16))
        self.caption_label.grid(row=self.grid_size, column=0, columnspan=self.grid_size, pady=10)

    def refresh_ui(self):
        for i, btn in enumerate(self.buttons):
            btn.config(text=str(self.tiles[i]) if self.tiles[i] else "")

    def swap_tiles(self, index):
        """Swap the empty tile with the tile at the given index."""
        self.tiles[self.empty_pos], self.tiles[index] = self.tiles[index], self.tiles[self.empty_pos]
        self.empty_pos = index  # Update the empty position
        self.refresh_ui()

    def move_left(self):
        empty_row, empty_col = divmod(self.empty_pos, self.grid_size)
        if empty_col > 0:  # Can move left if not in the leftmost column
            self.swap_tiles(self.empty_pos - 1)

    def move_right(self):
        empty_row, empty_col = divmod(self.empty_pos, self.grid_size)
        if empty_col < self.grid_size - 1:  # Can move right if not in the rightmost column
            self.swap_tiles(self.empty_pos + 1)

    def move_down(self):
        empty_row, empty_col = divmod(self.empty_pos, self.grid_size)
        if empty_row < self.grid_size - 1:  # Can move down if not in the bottom row
            self.swap_tiles(self.empty_pos + self.grid_size)

    def move_up(self):
        empty_row, empty_col = divmod(self.empty_pos, self.grid_size)
        if empty_row > 0:  # Can move up if not in the top row
            self.swap_tiles(self.empty_pos - self.grid_size)

    def move_tile(self, index):
        """Move a tile if it's adjacent to the empty tile."""
        row, col = divmod(index, self.grid_size)
        empty_row, empty_col = divmod(self.empty_pos, self.grid_size)

        if abs(row - empty_row) + abs(col - empty_col) == 1:  # Adjacent to empty space
            self.swap_tiles(index)

    def bind_keys(self):
        self.frame.bind("<Right>", lambda event: self.move_right())
        self.frame.bind("<Left>", lambda event: self.move_left())
        self.frame.bind("<Up>", lambda event: self.move_up())
        self.frame.bind("<Down>", lambda event: self.move_down())

def show():
    frame = tk.Tk()
    puzzle = SlidingPuzzle(frame)
    frame.mainloop()

def show(moves):
    frame = tk.Tk()
    puzzle = SlidingPuzzle(frame, caption_text=moves)
    puzzle.caption_text = moves
    frame.mainloop()

In [ ]:
show(a_star(root)[1::2])

The answer was found
